Import Libraries

In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
from scipy import stats

Load Data

In [2]:
orders = pd.read_csv('/content/List of Orders.csv')
details = pd.read_csv('/content/Order Details.csv')

Merge and prepare data

In [3]:
merged = orders.merge(details, on="Order ID", how="left")
merged["converted"] = (merged["Amount"] > 0).astype(int)
rev = merged.groupby("Order ID")["Amount"].sum().reset_index()
rev.rename(columns={"Amount": "revenue"}, inplace=True)
ab_data = orders[["Order ID", "Customer Name"]].merge(rev, on="Order ID", how="left")
ab_data["converted"] = (ab_data["revenue"] > 0).astype(int)

Random A/B assignment (since real variant not present)

In [4]:
np.random.seed(42)
ab_data["group"] = np.where(np.random.rand(len(ab_data)) < 0.5, "A", "B")

Conversion Analysis

In [5]:
nA = (ab_data.group=="A").sum()
nB = (ab_data.group=="B").sum()
xA = ab_data.loc[ab_data.group=="A","converted"].sum()
xB = ab_data.loc[ab_data.group=="B","converted"].sum()
pA = xA/nA
pB = xB/nB

In [6]:
stat, pval = proportions_ztest([xB, xA], [nB, nA], alternative="two-sided")
lift = pB/pA - 1 if pA>0 else np.nan

/usr/local/lib/python3.12/dist-packages/statsmodels/stats/weightstats.py:792: RuntimeWarning: invalid value encountered in scalar divide
  zstat = value / std


In [7]:
print("=== Conversion Analysis ===")
print(f"Group A: n={nA}, conversions={xA}, rate={pA:.4f}")
print(f"Group B: n={nB}, conversions={xB}, rate={pB:.4f}")
print(f"Relative lift: {lift:.2%}")
print(f"z-stat={stat:.3f}, p-value={pval:.3f}")

=== Conversion Analysis ===
Group A: n=241, conversions=241, rate=1.0000
Group B: n=259, conversions=259, rate=1.0000
Relative lift: 0.00%
z-stat=nan, p-value=nan


Revenue Analysis

In [8]:
revA = ab_data.loc[ab_data.group=="A","revenue"]
revB = ab_data.loc[ab_data.group=="B","revenue"]
t_stat, p_rev = stats.ttest_ind(revA, revB, equal_var=False)
meanA, meanB = revA.mean(), revB.mean()
lift_rev = meanB/meanA - 1 if meanA>0 else np.nan

In [9]:
print("\n=== Revenue Analysis ===")
print(f"Mean(A)={meanA:.2f}, Mean(B)={meanB:.2f}")
print(f"Relative lift: {lift_rev:.2%}")
print(f"t-stat={t_stat:.3f}, p-value={p_rev:.3f}")


=== Revenue Analysis ===
Mean(A)=823.12, Mean(B)=900.12
Relative lift: 9.35%
t-stat=-0.777, p-value=0.438
